In [2]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.optimizers import Adam
import numpy as np
import os
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import pickle

base_path = r"/Users/shravanisajekar/Downloads/27-3-2024 UPDATE"

def BCDU_net_D3(input_tensor=None):
    if input_tensor is None:
        inputs = Input(shape=(256, 256, 3))
    else:
        inputs = input_tensor

    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conc1 = concatenate([inputs, conv1], axis=3)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conc1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conc2 = concatenate([pool1, conv2], axis=3)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conc2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conc3 = concatenate([pool2, conv3], axis=3)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conc3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conc4 = concatenate([pool3, conv4], axis=3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conc4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    return model

def dataset_normalized(imgs):
    imgs_normalized = np.empty(imgs.shape)
    imgs_std = np.std(imgs)
    imgs_mean = np.mean(imgs)
    imgs_normalized = (imgs-imgs_mean)/imgs_std
    for i in range(imgs.shape[0]):
        imgs_normalized[i] = ((imgs_normalized[i] - np.min(imgs_normalized[i])) / (np.max(imgs_normalized[i])-np.min(imgs_normalized[i])))*255
    return imgs_normalized

tr_data = np.load(os.path.join(base_path, 'data_train.npy'))
te_data = np.load(os.path.join(base_path, 'data_test.npy'))
val_data = np.load(os.path.join(base_path, 'data_val.npy'))

tr_mask = np.load(os.path.join(base_path, 'mask_train.npy'))
te_mask = np.load(os.path.join(base_path, 'mask_test.npy'))
val_mask = np.load(os.path.join(base_path, 'mask_val.npy'))

tr_mask = np.expand_dims(tr_mask, axis=3)
te_mask = np.expand_dims(te_mask, axis=3)
val_mask = np.expand_dims(val_mask, axis=3)

print('ISIC18 Dataset loaded')

tr_data = dataset_normalized(tr_data)
te_data = dataset_normalized(te_data)
val_data = dataset_normalized(val_data)

tr_mask = tr_mask /255.
te_mask = te_mask /255.
val_mask = val_mask /255.

print('dataset Normalized')

input_tensor = Input(shape=(256, 256, 3))
model = BCDU_net_D3(input_tensor=input_tensor)
model.summary()
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy')

print('Training')
batch_size = 10
nb_epoch = 100

mcp_save = ModelCheckpoint('weight_isic18', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

history = model.fit(tr_data, tr_mask,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    shuffle=True,
                    validation_data=(val_data, val_mask),
                    callbacks=[mcp_save, reduce_lr_loss])
print('Trained model saved')
with open('hist_isic18', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)


ISIC18 Dataset loaded
dataset Normalized
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 256, 256, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 concatenate_8 (Concatenate  (None, 256, 256, 67)         0         ['input_2[0][0]',             
 )                                                                   'conv2d_19[0][0]']           
                                                                                                  
 conv2d_20 (Conv2D)          (None, 256, 256, 64)  

 e)                                                                  'conv2d_20[0][0]']           
                                                                                                  
 conv2d_35 (Conv2D)          (None, 256, 256, 64)         55360     ['concatenate_15[0][0]']      
                                                                                                  
 conv2d_36 (Conv2D)          (None, 256, 256, 64)         36928     ['conv2d_35[0][0]']           
                                                                                                  
 conv2d_37 (Conv2D)          (None, 256, 256, 1)          65        ['conv2d_36[0][0]']           
                                                                                                  
Total params: 24378529 (93.00 MB)
Trainable params: 24378529 (93.00 MB)
Non-trainable params: 0 (0.00 Byte)
__________________________________________________________________________________________________


Training
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 7.2632INFO:tensorflow:Assets written to: weight_isic18/assets


INFO:tensorflow:Assets written to: weight_isic18/assets


1/1 [==============================] - 7s 7s/step - loss: 7.2632 - val_loss: 30.6638 - lr: 0.0010
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 29.1174 - val_loss: 155.5737 - lr: 0.0010
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 188.8759INFO:tensorflow:Assets written to: weight_isic18/assets


...

1/1 [==============================] - 8s 8s/step - loss: 0.2686 - val_loss: 0.1820 - lr: 0.0010
Epoch 14/100
1/1 [==============================] - ETA: 0s - loss: 0.3062INFO:tensorflow:Assets written to: weight_isic18/assets


INFO:tensorflow:Assets written to: weight_isic18/assets


1/1 [==============================] - 7s 7s/step - loss: 0.3062 - val_loss: 0.1638 - lr: 0.0010
Epoch 15/100
1/1 [==============================] - 2s 2s/step - loss: 0.2968 - val_loss: 0.1708 - lr: 0.0010
Epoch 16/100
1/1 [==============================] - 2s 2s/step - loss: 0.2452 - val_loss: 0.4488 - lr: 0.0010
Epoch 17/100
1/1 [==============================] - ETA: 0s - loss: 0.3404INFO:tensorflow:Assets written to: weight_isic18/assets


INFO:tensorflow:Assets written to: weight_isic18/assets


1/1 [==============================] - 6s 6s/step - loss: 0.3404 - val_loss: 0.1333 - lr: 0.0010
Epoch 18/100
1/1 [==============================] - 1s 1s/step - loss: 0.2469 - val_loss: 0.1516 - lr: 0.0010
Epoch 19/100
1/1 [==============================] - 2s 2s/step - loss: 0.2840 - val_loss: 0.1549 - lr: 0.0010
Epoch 20/100
1/1 [==============================] - 2s 2s/step - loss: 0.2670 - val_loss: 0.2899 - lr: 0.0010
Epoch 21/100
1/1 [==============================] - 2s 2s/step - loss: 0.2631 - val_loss: 0.2615 - lr: 0.0010
Epoch 22/100
1/1 [==============================] - 3s 3s/step - loss: 0.2503 - val_loss: 0.1479 - lr: 0.0010
Epoch 23/100
1/1 [==============================] - 3s 3s/step - loss: 0.2436 - val_loss: 0.1351 - lr: 0.0010
Epoch 24/100
1/1 [==============================] - ETA: 0s - loss: 0.2468
Epoch 24: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
1/1 [==============================] - 3s 3s/step - loss: 0.2468 - val_loss: 0.1697 - lr: 